In [4]:
import xarray as xr
import rioxarray
import rasterio

# ==== User settings ====
input_nc = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\updated_viirs_binary_fire_2015_2016.nc"
output_tif = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\VIIRS_fire_label_2015_2016_30m.tif"
variable_name = 'fire_label'  # VIIRS variable name in .nc file
target_resolution = 30  # meters

# Choose an appropriate projected CRS (replace with your area's UTM zone if known)
target_crs = "EPSG:32644"

# Open dataset
ds = xr.open_dataset(input_nc)

# Extract the variable
da = ds[variable_name]

# 🔹 Instead of picking only first timestep, combine all timesteps
if 'time' in da.dims or 'valid_time' in da.dims:
    time_dim = 'time' if 'time' in da.dims else 'valid_time'
    # If fire occurred anytime → 1
    da = da.max(dim=time_dim)

# Write original CRS (assuming WGS84 lat/lon)
da = da.rio.write_crs("EPSG:4326")

# Reproject to target CRS and 30 m resolution
da_utm = da.rio.reproject(
    target_crs,
    resolution=target_resolution,
    resampling=rasterio.enums.Resampling.nearest  # nearest for classes
)

# Save to GeoTIFF
da_utm.rio.to_raster(output_tif)

print(f"✅ VIIRS raster saved at: {output_tif}")


✅ VIIRS raster saved at: C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\VIIRS_fire_label_2015_2016_30m.tif


In [5]:
import rioxarray
import numpy as np

# Path to your VIIRS 30m raster
input_tif = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\VIIRS_fire_label_2015_2016_30m.tif"

# Open raster
da = rioxarray.open_rasterio(input_tif, masked=True)

# Get NoData value from metadata
nodata_val = da.rio.nodata
print(f"📌 File: {input_tif}")
print(f"Detected NoData value in metadata: {nodata_val}")

# Get dimensions
height = da.sizes["y"]
width = da.sizes["x"]
bands = da.sizes["band"]

# Total pixels
total_pixels = height * width * bands

# Count missing pixels
if nodata_val is None:
    missing_pixels = np.isnan(da.values).sum()
else:
    missing_pixels = np.isnan(da.values).sum() + np.sum(da.values == nodata_val)

missing_percentage = (missing_pixels / total_pixels) * 100

print(f"Total pixels: {total_pixels}")
print(f"Missing pixels: {missing_pixels}")
print(f"Missing percentage: {missing_percentage:.2f}%")


📌 File: C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\VIIRS_fire_label_2015_2016_30m.tif
Detected NoData value in metadata: nan
Total pixels: 161942016
Missing pixels: 4582882
Missing percentage: 2.83%


In [ ]:
import rioxarray as rxr
import numpy as np
from scipy.ndimage import distance_transform_edt
from rasterio.enums import Resampling

# ==== Paths ====
input_tif = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\VIIRS_fire_label_2015_2016_30m.tif"
output_filled_tif =r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\VIIRS_fire_label_2015_2016_30m_filled.tif"

# ---- Load (masked=True makes NoData -> NaN in .values) ----
da = rxr.open_rasterio(input_tif, masked=True)  # shape: (band, y, x)
arr = da.values.astype(np.float32)              # ensure float for NaNs

# ---- Stats before ----
total = arr.size
missing_before = np.isnan(arr).sum()
print(f"Before fill: {missing_before} / {total} ({missing_before/total*100:.2f}%)")

def fill_nearest_nan_2d(a2d: np.ndarray) -> np.ndarray:
    """
    For a 2D array with NaNs, fill NaNs by copying the value from the nearest
    non-NaN pixel (Euclidean distance in pixel space).
    """
    data = a2d.copy()
    nan_mask = np.isnan(data)
    if not nan_mask.any():
        return data  # nothing to fill

    # distance_transform_edt on 'nan_mask' returns indices of nearest False (i.e., nearest valid)
    # We want indices of nearest valid cells for every pixel
    indices = distance_transform_edt(nan_mask, return_distances=False, return_indices=True)
    nearest_y = indices[0]
    nearest_x = indices[1]

    # Copy nearest valid values into NaN locations
    data[nan_mask] = data[nearest_y[nan_mask], nearest_x[nan_mask]]
    return data

# ---- Fill per band (works for single/multi-band) ----
filled = arr.copy()
for b in range(filled.shape[0]):
    filled[b] = fill_nearest_nan_2d(filled[b])

# ---- Build output DataArray & save ----
filled_da = da.copy()
filled_da.values = filled

# Remove nodata in output since we've filled it; keep same grid/CRS
filled_da = filled_da.rio.write_nodata(None)
filled_da.rio.to_raster(
    output_filled_tif,
    compress="LZW",  # keep file size reasonable
    tiled=True,
)

# ---- Verify ----
check = rxr.open_rasterio(output_filled_tif, masked=True).values
missing_after = np.isnan(check).sum()
print(f"After fill:  {missing_after} / {check.size} ({missing_after/check.size*100:.2f}%)")
print(f"✅ Saved: {output_filled_tif}")


Before fill: 4582882 / 161942016 (2.83%)


In [1]:
import rasterio
from rasterio.enums import Resampling
import numpy as np

# Input/output paths
input_file = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\VIIRS_fire_label_2015_2016_30m.tif"
output_file = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\VIIRS_fire_label_2015_2016_30m_filled.tif"

# Open dataset
with rasterio.open(input_file) as src:
    profile = src.profile
    data = src.read(1)  # read first band

    # Identify missing pixels
    nodata_val = src.nodata
    mask = np.isnan(data) if nodata_val is None else (data == nodata_val)

    print("Original missing pixels:", mask.sum())

    # Reproject to itself with nearest neighbor resampling → fills gaps
    filled_data = src.read(
        out_shape=src.shape,
        resampling=Resampling.nearest
    )[0]

    # Copy metadata and save new raster
    profile.update(dtype=rasterio.float32, compress="lzw")

    with rasterio.open(output_file, "w", **profile) as dst:
        dst.write(filled_data.astype(rasterio.float32), 1)

print("Filled raster saved:", output_file)


Original missing pixels: 4582882
Filled raster saved: C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\VIIRS_fire_label_2015_2016_30m_filled.tif


In [4]:
import rasterio
with rasterio.open(r"C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack_frp.tif") as src:
    profile =src.profile

In [5]:
profile

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -9999.0, 'width': 17, 'height': 13, 'count': 17521, 'crs': CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'), 'transform': Affine(0.2352923529411759, 0.0, 77.00002,
       0.0, -0.23075769230769247, 31.49988), 'blockxsize': 17, 'blockysize': 1, 'tiled': False, 'interleave': 'pixel'}